# Post-processing of case studies

## Usage notes

All figures will be written to the directory `img` as pdf and png files.

I run this notebook on Ubuntu, using this version of matplotlib:
```
pip freeze | grep matplotlib
matplotlib==2.1.0
```
Some older versions return an error because they do not support some plot configurations.
I set in the virtual machine the RAM to 12GB as 8GB is not sufficient to parse two annual result files.

## Import required libraries

In [1]:
import os
import cases

import matplotlib.pyplot as plt

from buildingspy.io.outputfile import Reader
from buildingspy.io.postprocess import Plotter

## Configure plots

In [2]:
plt.rcParams['axes.facecolor']='whitesmoke'
plt.rcParams['font.size'] = 6
plt.rcParams['text.usetex'] = False
plt.rcParams['legend.facecolor'] = 'white'
plt.rcParams['legend.framealpha'] = 0.75
plt.rcParams['legend.edgecolor'] = 'none'

def save_plot(figure, file_name):
    """ Save the figure to a pdf and png file in the directory `img`
    """
    import os
    out_dir = "img"
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    figure.savefig(os.path.join(out_dir, '{}.pdf'.format(file_name)))
    figure.savefig(os.path.join(out_dir, '{}.png'.format(file_name)))
    

def configure_axes(axes):
    """ Configure the axis style
    """
    axes.spines['right'].set_visible(False)
    axes.spines['top'].set_visible(False)
    axes.spines['left'].set_visible(False)
    axes.spines['bottom'].set_visible(False)
    axes.grid(color='grey', linewidth=0.25)
    return

## Read data of short-term simulations

In [3]:
winter=0
spring=1
summer=2
r_base=list()
r_base.insert(winter, Reader(cases.get_result_file_name('winter_base'), "dymola"))
r_base.insert(spring, Reader(cases.get_result_file_name('spring_base'), "dymola"))
r_base.insert(summer, Reader(cases.get_result_file_name('summer_base'), "dymola"))
print("Read base short simulations")

r_g36=list()
r_g36.insert(winter, Reader(cases.get_result_file_name('winter_g36'), "dymola"))
r_g36.insert(spring, Reader(cases.get_result_file_name('spring_g36'), "dymola"))
r_g36.insert(summer, Reader(cases.get_result_file_name('summer_g36'), "dymola"))
r_g36_noFreezeControl = Reader(cases.get_result_file_name('winter_g36_noFreezeControl'), "dymola")

print("Read G36 short simulations")

Read base short simulations
Read G36 short simulations


## Plot results

### Room temperatures

In [4]:
def plot_room_temp(readers):
    ''' Main method that plots the results
    '''
    # ------------------------------------------------------
    from buildingspy.io.outputfile import Reader
    import matplotlib.pyplot as plt
    import matplotlib

    # Optionally, change fonts to use LaTeX fonts
    #from matplotlib import rc
    #rc('text', usetex=True)
    #rc('font', family='serif')

    room_styles = ['k', 'b', 'g', 'r', 'c', 'g']
    room_names = ['nor', 'wes', 'sou', 'eas', 'cor']  
    plt.clf()   
    time_scale=86400.
    # ------------------------------------------------------

    # Read results
    for i in [0, 1, 2]:
        (t, TOut) = readers[i].values("weaBus.TDryBul")
        t = t/time_scale
        
        TRoo = list()
        reader=readers[i]
        
        for iRoo in range(len(room_names)):
            TRoo.insert(iRoo, reader.values(\
                "flo.{}.air.vol.T".format(room_names[iRoo]))[1])
    
        TSetHea = reader.values("conVAVCor.TRooHeaSet")[1]
        TSetCoo = reader.values("conVAVCor.TRooCooSet")[1]
   
        # Plot figure
        #plt.subplot(2, 1, 1)
        ax = plt.subplot(3, 1, i+1)

        ax.plot(t, TOut-273.15,    'gray', label='$T_{out}$', linewidth=0.25)
        for iRoo in range(len(room_names)):
            ax.plot(t, TRoo[iRoo]-273.15, room_styles[iRoo], \
                    label='$T_{{{name}}}$'.format(name=room_names[iRoo]), linewidth=0.25)
    
        ax.patch.set_facecolor('mistyrose')    
        ax.fill_between(t, TSetHea-273.15, y2=TSetCoo-273.15, color='whitesmoke')
    
        ax.set_xlabel('time [days]')
        ax.set_ylabel('temperature [$^\circ$C]')
    #ax.xaxis.set_ticks(np.arange(min(t)+0, 365, 1))

        ax.set_xlim([min(t)+5, min(t)+12])
                
        ax.legend(loc='center right', ncol=1)

        configure_axes(ax)

    return plt

# Create the plots
fig = plot_room_temp(r_base)
save_plot(fig, "TRoom_base")
fig = plot_room_temp(r_g36)
save_plot(fig, "TRoom_g36")

### VAV control

In [5]:
def plot_vav(readers):
    ''' Main method that plots the results
    '''
    # ------------------------------------------------------
    from buildingspy.io.outputfile import Reader
    import matplotlib.pyplot as plt

    room_styles = ['k', 'b', 'g', 'r', 'c', 'g']
    room_names = ['nor', 'wes', 'sou', 'eas', 'cor']  
    plt.clf()   
    time_scale=86400.
    # ------------------------------------------------------

    # Read results
    for i in [0, 1, 2]:
        (t, TOut) = readers[i].values("weaBus.TDryBul")
        t = t/time_scale
        
        yHea = list()
        yVAV = list()
        
        reader=readers[i]
        
        for kRoo in range(len(room_names)):
            yHea.insert(kRoo, reader.values(\
                "{}.yVal".format(room_names[kRoo]))[1])
            yVAV.insert(kRoo, reader.values(\
                "{}.yVAV".format(room_names[kRoo]))[1])
            
        # Plot figure
        #plt.subplot(2, 1, 1)
        ax = plt.subplot(3, 1, i+1)

        for iRoo in [0, 2]:
            ax.plot(t, yHea[iRoo], room_styles[iRoo], \
                   label='$y_{{hea,{name}}}$'.format(name=room_names[iRoo]), linewidth=0.25)
            ax.plot(t, yVAV[iRoo], room_styles[iRoo+1], \
                    label='$y_{{vav,{name}}}$'.format(name=room_names[iRoo]), linewidth=0.25)
        
        ax.set_xlabel('time [days]')
        ax.set_ylabel('control signal [$1$]')

        ax.set_xlim([min(t)+5, min(t)+12])
        ax.set_ylim([-0.05, 1.05])

 
        ax.legend(loc='center right', ncol=1)

        configure_axes(ax)

    return plt

# Create the plots
fig = plot_vav(r_base)
save_plot(fig, "vav_base")
fig = plot_vav(r_g36)
save_plot(fig, "vav_g36")

### AHU temperatures

In [6]:
def plot_ahu_temp(readers):
    ''' Main method that plots the results
    '''
    # ------------------------------------------------------
    from buildingspy.io.outputfile import Reader
    import matplotlib.pyplot as plt

    room_styles = ['k', 'b', 'g', 'r', 'c', 'g']
    room_names = ['nor', 'wes', 'sou', 'eas', 'cor']  
    plt.clf()   
    time_scale=86400.
    # ------------------------------------------------------

    # Read results
    for i in [0, 1, 2]:
        (t, TOut) = readers[i].values("weaBus.TDryBul")
        t = t/time_scale
        
        reader=readers[i]
        
        TSup = reader.values("TSup.T")[1]
        TMix = reader.values("TMix.T")[1]
        TRet = reader.values("TRet.T")[1]
  
        # Plot figure
        #plt.subplot(2, 1, 1)
        ax = plt.subplot(3, 1, i+1)

        ax.plot(t, TOut-273.15,    'gray', label='$T_{out}$', linewidth=0.25)
        ax.plot(t, TSup-273.15,    'r', label='$T_{sup}$', linewidth=0.25)
        ax.plot(t, TMix-273.15,    'g', label='$T_{mix}$', linewidth=0.25)
        ax.plot(t, TRet-273.15,    'b', label='$T_{ret}$', linewidth=0.25)

        
        ax.set_xlabel('time [days]')
        ax.set_ylabel('temperature [$^\circ$C]')
    #ax.xaxis.set_ticks(np.arange(min(t)+0, 365, 1))

        ax.set_xlim([min(t)+5, min(t)+12])
 
        ax.legend(loc='center right', ncol=1)

        configure_axes(ax)

    return plt

# Create the plots
fig = plot_ahu_temp(r_base)
save_plot(fig, "TAHU_base")
fig = plot_ahu_temp(r_g36)
save_plot(fig, "TAHU_g36")

Mixed air temperature with and without freeze protection

In [7]:
def plot_ahu_temp(readers):
    ''' Main method that plots the results
    '''
    # ------------------------------------------------------
    from buildingspy.io.outputfile import Reader
    import matplotlib.pyplot as plt

    plt.clf()   
    time_scale=86400.
    # ------------------------------------------------------

    # Read results
    
    for i in [0, 1]:
        (t, TOut) = readers[i].values("weaBus.TDryBul")
        t = t/time_scale
        
        reader=readers[i]
        
        if i == 0:
            (tWith, TMixWith) = reader.values("TMix.T")
            yEcoOutWith = reader.values("eco.yOut")[1]
            
            tWith = tWith/time_scale
        else:
            TMixNoFr = reader.values("TMix.T")[1]
            yEcoOutNoFr = reader.values("eco.yOut")[1]
            TSupNoFr = reader.values("TSup.T")[1]

  
    # Plot figure

    ax = plt.subplot(2, 1, 1)
    ax.plot(t, TOut-273.15,        'gray', label='$T_{out}$', linewidth=0.25)
    ax.plot(t, TSupNoFr-273.15,    'b', label='$T_{sup}$', linewidth=0.25)
    ax.plot(tWith, TMixWith-273.15,    'g', label='$T_{mix,with}$', linewidth=0.5)
    ax.plot(t, TMixNoFr-273.15,    'r', label='$T_{mix}$',  linewidth=0.5)

    #ax.set_xlabel('time [days]')
    ax.set_ylabel('temperature [$^\circ$C]')
    #ax.xaxis.set_ticks(np.arange(min(t)+0, 365, 1))

    ax.set_xlim([min(t)+5, min(t)+12])
 
    ax.legend(loc='center right', ncol=1)

    configure_axes(ax)

    ax = plt.subplot(2, 1, 2)
    ax.plot(tWith, yEcoOutWith,    'g', label='$y_{eco,out,with}$', linewidth=0.25)
    ax.plot(t, yEcoOutNoFr,    'r', label='$y_{eco,out,no}$',  linewidth=0.25)
    ax.set_xlabel('time [days]')
    ax.set_ylabel('$y_{eco,out}$ [$1$]')
    #ax.xaxis.set_ticks(np.arange(min(t)+0, 365, 1))

    ax.set_xlim([min(t)+5, min(t)+12])
 
    ax.legend(loc='center right', ncol=1)

    configure_axes(ax)
    
    
    return plt

# Create the plots
fig = plot_ahu_temp([r_g36[winter], r_g36_noFreezeControl])
save_plot(fig, "TMixFre")


### Flow rates and fan speed

In [8]:
def plot_flow_signals(readers):
    ''' Main method that plots the results
    '''
    # ------------------------------------------------------
    from buildingspy.io.outputfile import Reader
    import matplotlib.pyplot as plt

    room_styles = ['k', 'b', 'g', 'r', 'c', 'g']
    room_names = ['nor', 'wes', 'sou', 'eas', 'cor']  
    plt.clf()   
    time_scale=86400.
    # ------------------------------------------------------

    # Read results
    for i in [0, 1, 2]:
        (t, TOut) = readers[i].values("weaBus.TDryBul")
        t = t/time_scale
        
        reader=readers[i]
        
        yFanSup = reader.values("fanSup.y")[1]
        #yFanRet = reader.values("fanRet.y")[1]
        yEcoOut = reader.values("eco.yOut")[1]
        yEcoRet = reader.values("eco.yOut")[1]
  
        # Plot figure
        #plt.subplot(2, 1, 1)
        ax = plt.subplot(3, 1, i+1)

        ax.plot(t, yFanSup,    'r', label='$y_{fan,sup}$', linewidth=0.25)
        #ax.plot(t, yFanRet,    'g', label='$y_{fan,ret}$', linewidth=0.25)
        ax.plot(t, yEcoOut,       'g', label='$y_{eco,out}$', linewidth=0.25)
        ax.plot(t, yEcoRet,       'b', label='$y_{eco,ret}$', linewidth=0.25)

        
        ax.set_xlabel('time [days]')
        ax.set_ylabel('control signal [$1$]')
    #ax.xaxis.set_ticks(np.arange(min(t)+0, 365, 1))

        ax.set_xlim([min(t)+5, min(t)+12])
 
        ax.legend(loc='center right', ncol=1)

        configure_axes(ax)

    return plt

# Create the plots
fig = plot_flow_signals(r_base)
save_plot(fig, "flow_signals_base")
fig = plot_flow_signals(r_g36)
save_plot(fig, "flow_signals_g36")

### Normalized flow rates

In [9]:
def plot_normalized_flow_rates(readers):
    ''' Main method that plots the results
    '''
    # ------------------------------------------------------
    from buildingspy.io.outputfile import Reader
    import matplotlib.pyplot as plt

    room_styles = ['k', 'b', 'g', 'r', 'c', 'g']
    room_names = ['nor', 'wes', 'sou', 'eas', 'cor']  
    plt.clf()   
    time_scale=86400.
    # ------------------------------------------------------

    # Read results
    for i in [0, 1, 2]:
        (t, TOut) = readers[i].values("weaBus.TDryBul")
        t = t/time_scale
        
        reader=readers[i]
        
        mFanSup = reader.values("fanSup.m_flow")[1]/ \
            reader.values("fanSup.m_flow_nominal")[1][1]
#        mFanRet = reader.values("fanRet.m_flow")[1]/ \
#            reader.values("fanRet.m_flow_nominal")[1][1]
        mEco = reader.values("eco.port_Out.m_flow")[1]/ \
            reader.values("m_flow_nominal")[1][1]
  
        # Plot figure
        #plt.subplot(2, 1, 1)
        ax = plt.subplot(3, 1, i+1)

        ax.plot(t, mFanSup,    'r', label='$\dot m_{fan,sup}/\dot m_{0}$', linewidth=0.25)
#        ax.plot(t, mFanRet,    'g', label='$\dot m_{fan,ret}/\dot m_{0}$', linewidth=0.25)
        ax.plot(t, mEco,       'b', label='$\dot m_{eco,out}/\dot m_{0}$', linewidth=0.25)

        
        ax.set_xlabel('time [days]')
        ax.set_ylabel('normalized flow rates [$1$]')
    #ax.xaxis.set_ticks(np.arange(min(t)+0, 365, 1))

        ax.set_xlim([min(t)+5, min(t)+12])
        ax.set_ylim([0, 1])

 
        ax.legend(loc='center right', ncol=1)

        configure_axes(ax)

    return plt

# Create the plots
fig = plot_normalized_flow_rates(r_base)
save_plot(fig, "normalized_flow_base")
fig = plot_normalized_flow_rates(r_g36)
save_plot(fig, "normalized_flow_g36")

## Read annual results

This section also clears the old results to free up memory.

In [10]:
# Free up storage, then read new data
r_base = list()
r_g36 = list()
r_g36_annual=Reader(cases.get_result_file_name('annual_g36'), "dymola")
print("Read annual G36")
r_base_annual=Reader(cases.get_result_file_name('annual_base'), "dymola")
print("Read annual base")

Read annual G36
Read annual base


### Energy use

We convert the energy use to site electricity use, using an average COP for the chiller of 3.2, and a COP for heating of 4.0. These numbers are from https://portfoliomanager.energystar.gov/pdf/reference/Source%20Energy.pdf


In [11]:
COPc=3.2
COPh=4.0

In [12]:
def plot_energy(r_base, r_g36):
    from buildingspy.io.outputfile import Reader
    import matplotlib.pyplot as plt
    
    plt.clf()    

    # Conversion to kWh/m2
    conv = 1/3600./1000.
    readers = [r_base, r_g36]
    width = 0.5       # the width of the bars: can also be len(x) sequence

    hea    = [0., 0.]
    cooSen = [0., 0.]
    cooLat = [0., 0.]
    fan    = [0., 0.]
    cooLatBas = [0., 0.]
    coo = [0, 0]
    fanBas = [0., 0.]
    
    idx = [0, 1]
    for i in idx:
        hea[i]    =  readers[i].values('res.EHea')[1][-1] * conv / COPh
        cooSen[i] = -readers[i].values('res.ECooSen')[1][-1] * conv / COPc
        cooLat[i] = -readers[i].values('res.ECooLat')[1][-1] * conv / COPc
        coo[i] = cooSen[i] + cooLat[i]
        fan[i]    =  readers[i].values('res.EFan')[1][-1] * conv
        cooLatBas[i] = hea[i] + cooSen[i]
        fanBas[i]    = cooLatBas[i] + cooLat[i]
        
    p1 = plt.bar(idx, hea, width, color='r')
    p2 = plt.bar(idx, cooSen, width, bottom=hea, color='g')
    p3 = plt.bar(idx, cooLat, width, bottom=cooLatBas, color='b')
    p4 = plt.bar(idx, fan,    width, bottom=fanBas, color='k')
        
    plt.ylabel('site electricity use [kWh/(m2  a)]')
    plt.xticks([0, 1], ('base case', 'guideline 36'))
    plt.tick_params(axis=u'x', which=u'both',length=0)

    #plt.yticks(np.arange(0, 81, 10))
    plt.legend((p1[0], p2[0], p3[0], p4[0]), \
               ('heating', 'sensible cooling', 'latent cooling', 'fan'), \
              loc='upper right')
    
    
    save_plot(plt, "energy")
    
    # Write result to console and file
    eSit = [0, 0]
    for i in [0, 1]:
        eSit[i] = hea[i]+cooSen[i]+cooLat[i]+fan[i]

    str = """\
.. table:: Heating, cooling, fan and total site energy, and savings of guideline 36 case versus base case.

   ===================================== ===================================== ====================================== ====================================== =====
   :math:`E_{{h}} \quad [kWh/(m^2\,a)]`    :math:`E_{{c}} \quad [kWh/(m^2\,a)]`    :math:`E_{{f}} \quad [kWh/(m^2\,a)]`     :math:`E_{{tot}} \quad [kWh/(m^2\,a)]`     [%]
   ===================================== ===================================== ====================================== ====================================== =====
   {:37.4} {:37.4}  {:37.4}  {:37.4}   
   {:37.4} {:37.4}  {:37.4}  {:37.4}  {:4.3} 
   ===================================== ===================================== ====================================== ====================================== =====
  
  """.format(\
            hea[0], coo[0], fan[0], eSit[0], \
            hea[1], coo[1], fan[1], eSit[1], (1-eSit[1]/eSit[0])*100.)
    def save_rst(str, file_name):
        ''' Save the string `str` to the rst file `file_name.rst`
        '''
        print(str)
        with open(os.path.join("img", "{}.rst".format(file_name)), "w") as fil:
            fil.write(str)
        
    save_rst(str, "site_energy")
plot_energy(r_base_annual, r_g36_annual)

.. table:: Heating, cooling, fan and total site energy, and savings of guideline 36 case versus base case.

   ===================================== ===================================== ====================================== ====================================== =====
   :math:`E_{h} \quad [kWh/(m^2\,a)]`    :math:`E_{c} \quad [kWh/(m^2\,a)]`    :math:`E_{f} \quad [kWh/(m^2\,a)]`     :math:`E_{tot} \quad [kWh/(m^2\,a)]`     [%]
   ===================================== ===================================== ====================================== ====================================== =====
                                    4.46                                 19.66                                  3.563                                  27.68   
                                   3.722                                 14.93                                  1.861                                  20.51  25.9 
   ===================================== ===================================== 

### Diagnostic output

In [13]:
print("Average outdoor air intake, base {} kg/s".format(r_base_annual.mean('eco.port_Out.m_flow')))
print("Average outdoor air intake, G36  {} kg/s".format(r_g36_annual.mean( 'eco.port_Out.m_flow')))

print("CPUtime, base {} h".format(r_base_annual.values('CPUtime')[1][-1]/3600.))
print("CPUtime, G36  {} h".format(r_g36_annual.values( 'CPUtime')[1][-1]/3600.))

Average outdoor air intake, base 0.743576951588 kg/s
Average outdoor air intake, G36  0.664621322463 kg/s
CPUtime, base 1.83920857747 h
CPUtime, G36  2.88293375651 h


In [14]:
(t, y) = r_base_annual.values('flo.sou.air.vol.T')
plt.clf()
plt=Plotter.boxplot(t=t, y=y-273.15, increment=3600, nIncrement=24)

# Decorate, save and show the plot
plt.xlabel('Time [h]') 
plt.ylabel(u'Room temperature [$^\circ$C]') 
plt.grid()
plt.savefig("roomTemperatures.png")
plt.savefig("roomTemperatures.pdf")
plt.title('South zone, base case')
#plt.show() 

Text(0.5,1,u'South zone, base case')

## Energy use

In [15]:
2.8*60

168.0